#### Diving In

**Iterators** are everywhere in Python 3. The are the secret sauce, just out of sight. **Comprehensions** are just a simple form of *iterators*. **Generators** are just a simple form of *iterators*

A function that *yield*s values is a nice, compact way of building an iterator.

In [17]:
class Fib:
    '''iterator that yield numbers in the fibonacci sequence'''
    
    def __init__(self, max: int):
        self.max: int = max
    
    def __iter__(self):
        self.a: int = 0
        self.b: int = 1
        return self
    
    def __next__(self):
        fib: int = self.a
        if fib > self.max:
            raise StopIteration
            
        self.a, self.b = self.b, self.a + self.b
        return fib

In [19]:
list(Fib(1000))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987]

##### Defining Classes

Python is fully object-oriented: you can define your own classes, inherit from your own or built-in classes, and instantiate the classes you've defined.

Python does not have a separate _interface_ definition. Just define the class and start coding.

A python class starts with the reserved word `class`, followed by the class name.

The name of the class in `Fib` it does not inherit from any other class. Class naming convention uses camel case.

Class code is nested like other block statements such as `if`.

The `__init__` method (uses two underscores) is the constructor for the class, sometimes also called an initializer - but this is splitting hairs.

The first argument to every class method including the `__init__` method is always a reference to the current instance of the class. By *convention* this argument is called **self**. 

Unlike Java's **this** keyword, **self** in Python is not a keyword, but a very strong convention.

In all instance methods, **self** refers to the instance whose method was classed. Although you need to specify _self_ explicitly when defining the method you do not specify it when calling the method; Python will add it for you automatically.

**Sidebar**

    pass is a python keyword, that is used to leave implementation detail to a later time when you are stubbing out code.

    The pass statement in Python is like an empty set of `{}` curly braces in Java.

##### Instantiating Classes

Instantiation a class in Python is straightforward. To instantiate a class, simply call the class as if it were a function, passing the arguments that `__init__()` method requires. The return value will be the newly created object.

There is no explicit `new` operator in Python.

In [22]:
f = Fib(1000)
f

In [23]:
type(f)

__main__.Fib

In [24]:
f.__class__

__main__.Fib

In [59]:
f.__doc__

'iterator that yield numbers in the fibonacci sequence'

##### Instance Variables

What is `self.max` in `Fib`? It's an instance variable. It is completely seperate from max, which was passed into the `__init__()` method as an argument. The `max` passed into the init method is only available in the constructor. `self.max` is available to all instance methods in the class.


Instance variables are specific to one instance of a class. If you create two Fib instance with different maximum values, they will each have their own value.

In [61]:
f = Fib(1000)
g = Fib(2000)

In [62]:
print(f.max)
print(g.max)

1000
2000


##### Classes Vs Instances

Lets dive into class members vs instance members to understand the implementation and relationship between `classes` and `instances`.

In [79]:
class Car:
    engine = "combustion"
    cylinder = "4"

In [80]:
c1 = Car()
print(f"Car with {c1.engine} engine having {c1.cylinder} cylinders")

Car with combustion engine having 4 cylinders


When we create an instance of Car, think of it as instantiating an empty dictionary. Infact this is exactly how the implementation works and we can view the dictionary using the instance member / attribute `__dict__`

In [81]:
c1.__dict__

{}

The `type` of `c1` is `Car`. Since everything in Python is an object the `type Car` is an object itself and must have a `__dict__` attribute

In [82]:
type(c1)

__main__.Car

There are multiple ways to access the dictionary.

In [83]:
c1.__class__.__dict__

mappingproxy({'__module__': '__main__',
              'engine': 'combustion',
              'cylinder': '4',
              '__dict__': <attribute '__dict__' of 'Car' objects>,
              '__weakref__': <attribute '__weakref__' of 'Car' objects>,
              '__doc__': None})

In [84]:
Car.__dict__

mappingproxy({'__module__': '__main__',
              'engine': 'combustion',
              'cylinder': '4',
              '__dict__': <attribute '__dict__' of 'Car' objects>,
              '__weakref__': <attribute '__weakref__' of 'Car' objects>,
              '__doc__': None})

The instance `c1` inherits default values of `engine` and `cylinder` from the `type Car`, wich you can see in the output of the `__dict__` attribute of `Car`.

This would imply that we can post production change the value of th `engine` attribute of `c1` by changing the value of the attribute `Car`.

In [85]:
Car.engine = "electric"
Car.cylinder = 0
print(f"Car with {c1.engine} engine having {c1.cylinder} cylinders")

Car with electric engine having 0 cylinders


However this would be a **grave misunderstanding**. Python implement classes and object as **prototypes** similar to javascript and **unlike** Java, which implements templates.

The implication is the resolution used by the runtime to find an attribute. In Python, the runtime will look for an attribute in the instance dictionary, if not found will proceed to look for the value in the type dictionary.

Or type members can be thought of a providing default values. Lets demonstrate this with an example of a car with 8 cylinders

In [89]:
# reset default engine and cylinder for Car
Car.engine = 'combustion'
Car.cylinder = 4
c2 = Car()
c2.make = "Jaguar"
c2.cylinder = 8

In [90]:
c2.__dict__

{'make': 'Jaguar', 'cylinder': 8}

In [93]:
print(f"Car by {c2.make} with {c2.engine} engine having {c2.cylinder} cylinders")

Car by Jaguar with combustion engine having 8 cylinders


In [95]:
print(f"Car with {c1.engine} engine having {c1.cylinder} cylinders")

Car with combustion engine having 4 cylinders


Lets override `c1` cylinder to `12`, muscle cars for everyone

In [96]:
c1.cylinder = 12
print(f"Car with {c1.engine} engine having {c1.cylinder} cylinders")

Car with combustion engine having 12 cylinders


However this does not change c2

In [97]:
print(f"Car by {c2.make} with {c2.engine} engine having {c2.cylinder} cylinders")

Car by Jaguar with combustion engine having 8 cylinders


The key to understanding prototypes is the realization and understanding of the dynamic nature of the implementation. Behavior (`functions`) are shared across instances, members are not shared.

##### A Fibonacci Iterator

An _**iterable**_ object is an object that implements `__iter__`, which return an _**iterator**_ object.

An _**iterator**_ object is an object that implements `__next__`, which is expected to return the next element of the iterable object that returned the iterator. An _**iterator**_ raises a `StopIteration` exception when no more elements are available

In the simplest case the _**iterable**_ will implement `__next__` itself and return `self` in `__iter__`

You can use iterables in `for` loops and you can construct lists from them.

All three of the class methods, `__init__`, `__iter__`, `__next__` begin and end with a pair of underscore (\_) characters. There is nothing magical about it, but it usually indicates that these are _special methods_. The only thing "special" about special methods is that they aren't called directly; Python calls them when you use some other syntax on the class or an instance of the class.

In [100]:
class Fib2:
    '''iterator that yield numbers in the fibonacci sequence'''
    
    def __init__(self, max: int):
        self.max: int = max
    
    def __iter__(self):
        print("Invoking __iter__ method on this iterable, returning an iterator")
        self.a: int = 0
        self.b: int = 1
        return self
    
    def __next__(self):
        print("Invoking __next__ method on this iterator, returning a value")
        fib: int = self.a
        if fib > self.max:
            raise StopIteration
            
        self.a, self.b = self.b, self.a + self.b
        return fib

In [101]:
list(Fib2(1000))

Invoking __iter__ method on this iterable, returning an iterator
Invoking __next__ method on this iterator, returning a value
Invoking __next__ method on this iterator, returning a value
Invoking __next__ method on this iterator, returning a value
Invoking __next__ method on this iterator, returning a value
Invoking __next__ method on this iterator, returning a value
Invoking __next__ method on this iterator, returning a value
Invoking __next__ method on this iterator, returning a value
Invoking __next__ method on this iterator, returning a value
Invoking __next__ method on this iterator, returning a value
Invoking __next__ method on this iterator, returning a value
Invoking __next__ method on this iterator, returning a value
Invoking __next__ method on this iterator, returning a value
Invoking __next__ method on this iterator, returning a value
Invoking __next__ method on this iterator, returning a value
Invoking __next__ method on this iterator, returning a value
Invoking __next__ me

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987]

In [104]:
for n in Fib2(1000):
    print(n, end = '\n')

Invoking __iter__ method on this iterable, returning an iterator
Invoking __next__ method on this iterator, returning a value
0
Invoking __next__ method on this iterator, returning a value
1
Invoking __next__ method on this iterator, returning a value
1
Invoking __next__ method on this iterator, returning a value
2
Invoking __next__ method on this iterator, returning a value
3
Invoking __next__ method on this iterator, returning a value
5
Invoking __next__ method on this iterator, returning a value
8
Invoking __next__ method on this iterator, returning a value
13
Invoking __next__ method on this iterator, returning a value
21
Invoking __next__ method on this iterator, returning a value
34
Invoking __next__ method on this iterator, returning a value
55
Invoking __next__ method on this iterator, returning a value
89
Invoking __next__ method on this iterator, returning a value
144
Invoking __next__ method on this iterator, returning a value
233
Invoking __next__ method on this iterator, r

`Fib2` used in the `for` loop and `list` function is identical to how we called fibonacci as a generator. 

There's a bit of magic involved in for loops. Here's what happens

* The `for` loop calls `Fib2(1000)` the constructor. This returns an instance (`fib`) of `Fib` class. The instance is an `iterable`
* Secretly, the `for` loop calls `__iter__` method on the `iterable`, which returns an `iterator` object.
* To _loop through_ the iterator, the `for` loop calls `__next__` on the iterator, which computes the next fibonacci number,
* The `for` loop stops the iteration when a call to the `__next__` method result in a `StopIteration` exception. The for loop will swallow the exception and gracefully exit. Other exceptions will pass through and be raised as usual.